In [1]:
#import manager
#import minecart
import pandas as pd
from gc import collect
from collections import Counter
from IPython.display import Markdown

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, XMLConverter
from pdfminer.image import ImageWriter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO
from tabula import read_pdf
import tabula
# from PIL import Image

In [ ]:
def safe_float(x):
    try:
        return float(x)
    except:
        return np.nan
koshtak.loc[:, "No."] = koshtak["No."].apply(safe_float)

In [4]:
rsrcmgr = PDFResourceManager()
retstr = StringIO()
codec = "utf-8"
laparams = LAParams()
device = (TextConverter if "x"=="text" else XMLConverter)(rsrcmgr, retstr, codec=codec, laparams=laparams, imagewriter=ImageWriter("./images"))
fp = open(path, "rb")
interpreter = PDFPageInterpreter(rsrcmgr, device)
password = ""
btype="xml"
maxpages = 0
caching = True
pagenos = set()
from bs4 import BeautifulSoup as bs
for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
    interpreter.process_page(page)
fp.close()
device.close()
s = retstr.getvalue()
s = bs(s, "xml")
retstr.close()


In [7]:
imgs = [{"id": x.find_parent("page")["id"], "src": x["src"],
         "bbox": map(float, x.parent["bbox"].split(",")),
         "height": float(x["height"]), "width": float(x["width"])}
        for x in s.find_all("image")]
        #if float(x["height"]) > 50 and float(x["width"]) > 50]

In [58]:
lines = []
def get_size(t):
    return round(np.float(t.get("size") or .0))
for page in s.find_all("page"):
    imgs_t = [x for x in imgs
              if x["id"] == page["id"]]
    llines = []
    for line in page.find_all("textline"):
        d = dict.fromkeys(["text", "fonts", "sizes", "bbox"]) #
        d["bbox"] = map(float, line["bbox"].split(","))
        texts = line.find_all("text")
        text = "".join([x.text for x in texts])#.strip()
#         if len(llines) > 2 and llines[-2]["raw_text"].strip(" ") == "\n"\
#         and llines[-1]["raw_text"].strip(" ") == "\n" and text.strip(" ") == "\n":
#             text = ""
        if not text:
            continue
        d["text"] = text.strip().lower()
        d["raw_text"] = text
        d["fonts"] = [x.get("font") or "" for x in texts]
        d["sizes"] = [get_size(x) for x in texts]
        d["pageno"] = int(line.findParent("page")["id"])
        #print d["text"], d["bbox"]
        llines.append(d)
    for img in imgs_t:
        if img["height"] <= 50. and img["width"] <= 50.:
            continue
        try:
            y0s = [line["bbox"][1]
                   if line.has_key("bbox") else -10000
                   for line in llines]
            img_y1 = img["bbox"][3]
            i = 0
            diff = y0s[i] - img_y1
            while diff > 0 and i < len(y0s):
                i += 1
                if y0s[i] != -10000:
                    diff = y0s[i] - img_y1
            llines.insert(i, {
                "raw_text": "<img src='./images/%s' width='%.2f' height='%.2f'/>\n" % (
                    img["src"], img["width"], img["height"]),
                "pageno": int(page["id"])
            })
        except:
            print "---------kaik locha che--------\n"
    diffs, skips, useds = {}, [], []
#     for i in range(len(llines)):
#         if not llines[i].get("bbox") or i in skips:
#             continue
#         diffs[i] = []
#         for j in range(i + 1, len(llines)):
#             if not llines[j].get("bbox"): continue
#             absd = abs(llines[i]["bbox"][1] - llines[j]["bbox"][1])
#             #print absd, llines[i]["text"], llines[j]["text"]
#             if absd <= 6:
#                 skips.append(j)
#                 diffs[i].append(j)
#         if not diffs[i]:
#             diffs.pop(i)
#     for i in sorted(diffs.keys()):
#         currimgs = []
#         for j in imgs_t:
#             if abs(llines[i]["bbox"][1] - j["bbox"][1]) <= 10:
#                 currimgs.append(j)
#                 useds.append(j)
#         currstr = llines[i]["raw_text"].strip() + (" <img src='./images/%s'/>" % currimgs.pop(0)["src"] if currimgs else " ")
#         for j in diffs[i]:
#             currstr += llines[j]["raw_text"].strip() + ((" <img src='./images/%s'/>" % currimgs.pop(0)["src"]) if currimgs else " ") + " "
#         llines[i]["raw_text"] = currstr + "\n"
#         llines[i]["text"] = currstr.strip()
#         #print currstr + "\n"
#         #print llines[i]["raw_text"].strip() + " " + " ".join([ ])
#     for j in useds:
#         imgs_t.remove(j)
#     # for x in sorted([y for x in diffs.values() for y in x], reverse=True):
#     #     del llines[x]
#     for i in imgs_t:
#         for j in llines:
#             if j.has_key("bbox") and abs(i["bbox"][1] - j["bbox"][1]) <= 10:
#                 j["raw_text"] = "<img src='./images/%s'/> " % i["src"] + j["raw_text"]
    lines.extend(llines)


In [59]:
df_lines = pd.DataFrame.from_dict(lines).assign(
    freqsize=lambda x: x["sizes"].fillna("").apply(lambda y: Counter(y).most_common(1)[0][0] if y else 0.)).drop("sizes", 1).assign(
    freqfont=lambda x: x["fonts"].fillna("").apply(lambda y: Counter(y).most_common(1)[0][0] if y else "NA")).drop("fonts", 1)